In [35]:
from pandas import Index, Series, DataFrame, np; import pandas as pd
from toolz.curried.operator import *; from toolz.curried import *

In [36]:
from functools import wraps

class BaseIndex(Index):
    _base, _dtype = Index, object
    
    @property
    def _attr_mapper(self):
        yield tuple(dir(self._base)), Series
    
    def __new__(cls, data=None, dtype=None, copy=False, name=None, fastpath=False, tupleize_cols=True, **kwargs):
        if isiterable(data) and not isinstance(data, Index):
            try:
                data = cls._coerce_to_ndarray(map(cls._base, data))
            except (TypeError, ValueError): ...
            
        return cls._simple_new(data, name, dtype=dtype, copy=copy, fastpath=fastpath, tupleize_cols=tupleize_cols, **kwargs).astype(dtype or cls._dtype)
    
    def __getattribute__(self, name):
        try:
            return super().__getattribute__(name)
        except AttributeError as e:
            for attrs, cls in self._attr_mapper:
                if name in attrs: break
            else:
                raise e
            
            attr = getattr(self._base, name, None)
            
            # There is no way to interpret the attribute
            if cls is False: raise NotImplemented
            
            if isinstance(attr, property) or not callable(attr): #  return computed values, not a callable.
                result = cls(self.map(attrgetter(name)))
                if cls in (Series, DataFrame): 
                    result.index = self
                return (self if cls is None else result)

            # wrap a callable for later.
            @wraps(attr)
            def mapper(*args, **kwargs):                        
                if cls is None: return self        
                result = cls(self.map(partial(lambda v: attr(v, *args, **kwargs))))
                if cls in (Series, DataFrame): result.index = self
                return result
            return mapper
    
    def __dir__(self):
        """Update the available attribute."""
        return list(super().__dir__()) + dir(self._base) + list(concat(pluck(0, self._attr_mapper)))
    
    def astype(self, dtype, copy=True):
        if dtype: self._data = self._data.astype(dtype, copy=copy)
        return self